In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import copy
from scipy.optimize import minimize 
from scipy import stats
import random

sns.set()
mpl.use('pgf') # for latex

In [2]:
# (a): Exploring the data (6)

# read .csv
score = pd.read_csv("stai_scores.csv", header = None)[0]
choices = pd.read_csv("inst_choices.csv", header = None)
outcomes = pd.read_csv("inst_outcomes.csv", header = None)

# statistics
score_mean = np.mean(score)
score_std = np.std(score)
score_median = np.median(score)

# cutoff count
cutoff_count = sum(score <= 43)
cutoff_index = [i for i in range(len(score)) if score[i] <= 43]
cutoff_count_unfit = sum(score[25:50] > 43)

# choices & outcomes count
choices_count = []
for i in range(50):
    choices_count.append(sum(choices.loc[i] == 1))
choices_count_sub1 = choices_count[0] / 160
average_choices_percentage = np.mean(choices_count) / 160
random_aversive_sound = 160 * 0.25 * (0.6 + 0.8 + 0.6 + 0.65)

# output
print("Mean:", score_mean)
print("Standard deviation:", round(score_std, 2))
print("Median:", score_median)
print("Count of healthy controls (STAI≤43):", cutoff_count)
print("Indices of the control subjects:", cutoff_index)
print("Count of unfitted healthy controls:", cutoff_count_unfit)
print("Count of choosing A:", choices_count)
print("Average percentage of choosing A (subject 1):", round(choices_count_sub1, 2))
print("Average count of choosing A:", round(average_choices_percentage, 2))
print("Random strategy expected count of aversive sound:", round(random_aversive_sound))

Mean: 45.42
Standard deviation: 15.92
Median: 42.5
Count of healthy controls (STAI≤43): 25
Indices of the control subjects: [21, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Count of unfitted healthy controls: 1
Count of choosing A: [29, 40, 55, 15, 46, 36, 55, 39, 30, 33, 44, 27, 27, 32, 43, 39, 15, 23, 48, 48, 28, 41, 39, 24, 61, 33, 48, 47, 38, 22, 66, 40, 34, 46, 35, 51, 37, 33, 65, 50, 29, 26, 34, 55, 23, 58, 27, 28, 41, 22]
Average percentage of choosing A (subject 1): 0.18
Average count of choosing A: 0.24
Random strategy expected count of aversive sound: 106


In [3]:
# (b): Simulations (7)

# update strategy 1
def update_1(V, action, outcome, alpha):
    V[action-1] = V[action-1] + float(alpha) * (outcome - V[action-1])
    return V

# softmax decision
def softmax_decision(V, beta):
    p_action_A = 1 / (1 + np.exp(-beta * (V[1] - V[0])))
    return p_action_A

# simulation
def simulation(V0, n, update, decision, alpha, beta, sample = False):
    p_outcome_A1 = [0.6, 0.8, 0.6, 0.65]
    V_AB = np.zeros([161, 2])
    aver_stim = 0
    if sample == True:
        outcomes_sample = np.zeros(160)
        choices_sample = np.zeros(160)
    for s in range(n):
        V = copy.deepcopy(V0)
        V_AB[0,:] = V_AB[0,:] + np.array(V)
        for k in range(4):
            for i in range(40):
                # action
                if decision(V, beta) >= np.random.rand(1):
                    action = 1
                    p_outcome_1 = p_outcome_A1[k]
                else:
                    action = 2
                    p_outcome_1 = 1 - p_outcome_A1[k]
                # outcome
                if p_outcome_1 >= np.random.rand(1):
                    outcome = 1
                    aver_stim = aver_stim + 1
                else:
                    outcome = 0
                # update
                V = update(V, action, outcome, alpha)
                V_AB[k*40+i+1, :] = V_AB[k*40+i+1, :] + np.array(V)
                # record
                if sample == True:
                    choices_sample[k*40+i] = int(action)
                    outcomes_sample[k*40+i] = int(outcome)
    V_AB = V_AB / n
    aver_stim = aver_stim / n
    if sample == True:
        return choices_sample, outcomes_sample
    return V_AB, aver_stim

# config
V0 = [0.5, 0.5]
n = 10000
update = update_1
decision = softmax_decision
alpha = 0.3
beta = 8

# implement
V_AB_1, aver_stim_1 = simulation(V0, n, update, decision, alpha, beta)

# plot
fig, ax =  plt.subplots(1, 1, sharex = True, figsize = (6,3.7))
label = ['V(A)', 'V(B)', 'V(A)-V(B)']
color = ['#FFC75F', '#FF9671', '#FF6F91']
ax.plot(np.arange(0, 161, 1), V_AB_1[:, 0], label = label[0], color = color[0])
ax.plot(np.arange(0, 161, 1), V_AB_1[:, 1], label = label[1], color = color[1])
ax.plot(np.arange(0, 161, 1), V_AB_1[:, 0] - V_AB_1[:,1], label = label[2], \
        color = color[2])
ax.set_title('Simulation evolution (n=10000)')
ax.set_xlabel('Trials')
ax.set_ylabel('Average action values')
ax.legend(loc = 'upper right')
plt.show()

# check
print("Average number of aversive sounds:", aver_stim_1)

# image export
# plt.savefig('1.pgf', format='pgf')

Average number of aversive sounds: 60.872


<ipython-input-3-27bd0dce688c>:74: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [4]:
# (c): Exploring parameter settings (7)

# config
V0 = [0.5, 0.5]
n = 2
update = update_1
decision = softmax_decision
alpha = 0
beta0 = 0
intv = 0.05
sepn = int(1/intv) - 1

# implement
aver_stim = np.zeros([sepn,sepn])
for i in range(sepn):
    alpha = alpha + intv
    beta = beta0
    for j in range(sepn):
        beta = beta + 10 * intv
        __, aver_stim_ab = simulation(V0, n, update, decision, alpha, beta)
        aver_stim[i,j] = aver_stim_ab

# plot
fig = plt.figure(figsize = (6,4.7))
ax = fig.gca(projection = '3d')
aa = np.arange(intv, 1, intv)
bb = np.arange(10*intv, 10, 10*intv)
beta_axis, alpha_axis = np.meshgrid(bb, aa)
alpha_axis.resize(sepn*sepn)
beta_axis.resize(sepn*sepn)
aver_stim.resize(sepn*sepn)
plot_c = ax.plot_trisurf(beta_axis, alpha_axis, aver_stim, cmap = plt.cm.viridis, \
                         linewidth = 0)
fig.colorbar(plot_c)
ax.set_title('Average number of aversive stimuli (n=1000)')
ax.set_xlabel(r'$\beta$')
ax.set_ylabel(r'$\alpha$')
plt.show()

# image export
# plt.savefig('2.pgf', format='pgf')

<ipython-input-4-23ee5fc32610>:37: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [5]:
# (d): Likelihood function (6)

# negative log likelihood
def nll(theta, choices, outcomes, V0):
    length_num = len(choices)
    V = copy.deepcopy(V0)
    nll = np.zeros(length_num)
    for i in range(length_num):
        c = int(choices[i])
        c_alt = 1 + int(c==1)
        nll[i] = 1 / (1 + np.exp(-theta[1] * (V[c_alt-1] - V[c-1])))
        V[c-1] = V[c-1] + theta[0] * (int(outcomes[i]) - V[c-1])
    nll = -np.sum(np.log(nll))
    return nll

# config
V0 = [0.5, 0.5]
theta = [0.3, 8]

# implement
nll_1 = nll(theta, choices.loc[0], outcomes.loc[0], V0)
nll_2 = nll(theta, choices.loc[1], outcomes.loc[1], V0)
nll_10 = nll(theta, choices.loc[9], outcomes.loc[9], V0)

# output
print("NLL for 1st participant is:", round(nll_1,2))
print("NLL for 2nd participant is:", round(nll_2,2))
print("NLL for 10th participant is:", round(nll_10,2))

NLL for 1st participant is: 50.19
NLL for 2nd participant is: 64.24
NLL for 10th participant is: 58.6


In [6]:
# (e): Model fitting (9)

# optimization
V0 = [0.5, 0.5]
theta_opt = np.zeros([50,2])
nll_a_obj = np.zeros(50)
for i in range(50):    
    nll_result = minimize(nll, [0.3, 8], args = (choices.loc[i], outcomes.loc[i], \
                                                 V0), method = "Nelder-Mead")
    theta_opt[i, :] = nll_result['x']
    nll_a_obj[i] = nll_result['fun']

# statistics
theta_mean = np.mean(theta_opt, axis = 0)
theta_var = np.var(theta_opt, axis = 0)
theta_cor = np.corrcoef(theta_opt[:, 0], theta_opt[:, 1])[0, 1]
theta_cor_1 = np.corrcoef(theta_opt[0:25, 0], theta_opt[0:25, 1])[0, 1]
theta_cor_2 = np.corrcoef(theta_opt[25:50, 0], theta_opt[25:50, 1])[0, 1]

# plot
fig, ax =  plt.subplots(1, 1, sharex = True, figsize = (6,3.7))
label = [r'$\alpha$', r'0.1$\beta$']
color = ['#FF9671', '#FF6F91']
# color = ['#FF6F91', '#D65DB1']
for i in range(50):
    plt.vlines(x = i+1, ymin = theta_opt[i, 0], ymax = 0.1*theta_opt[i, 1], \
               color = '#FFC75F', zorder = 1)
plt.axvline(x = 25.5, ymin = 0, ymax = 1, linestyle = "dashed", \
            color = '#FFC75F', zorder = 2)
plt.axhline(y = np.mean(theta_opt[:, 0]), xmin = 0, xmax = 1, \
            linestyle = "dashed", color = color[0], zorder = 2)
plt.axhline(y = 0.1*np.mean(theta_opt[:, 1]), xmin = 0, xmax = 1, \
            linestyle = "dashed", color = color[1], zorder = 2)
plt.axhline(y = np.mean(theta_opt[0:25, 0]), xmin = 0, xmax = 0.5, \
            color = color[0], zorder = 2)
plt.axhline(y = 0.1*np.mean(theta_opt[0:25, 1]), xmin = 0, xmax = 0.5, \
            color = color[1], zorder = 2)
plt.axhline(y = np.mean(theta_opt[25:50, 0]), xmin = 0.5, xmax = 1, \
            color = color[0], zorder = 2)
plt.axhline(y = 0.1*np.mean(theta_opt[25:50, 1]), xmin = 0.5, xmax = 1, \
            color = color[1], zorder = 2)
ax.scatter(np.arange(1, 51, 1), theta_opt[:, 0], label = label[0], \
           color = color[0], zorder = 3)
ax.scatter(np.arange(1, 51, 1), 0.1*theta_opt[:, 1], label = label[1], \
           color = color[1], zorder = 3)
ax.set_title('Parameter optimization of NLL by Nelder-Mead')
ax.set_xlabel('Participant index')
ax.set_ylabel('Parameter value')
ax.legend(loc = 'upper right')
plt.show()

# output
print("Mean of fitted parameter values [alpha, beta]:", \
      [round(i,2) for i in theta_mean])
print("Variance of fitted parameter values [alpha, beta]:", \
      [round(i,2) for i in theta_var])
print("Pearson’s correlation coefficient (total):", round(theta_cor,2))
print("Pearson’s correlation coefficient (group 1):", round(theta_cor_1,2))
print("Pearson’s correlation coefficient (group 2):", round(theta_cor_2,2))

# image export
# plt.savefig('3.pgf', format='pgf')

Mean of fitted parameter values [alpha, beta]: [0.42, 5.28]
Variance of fitted parameter values [alpha, beta]: [0.01, 3.04]
Pearson’s correlation coefficient (total): 0.07
Pearson’s correlation coefficient (group 1): -0.13
Pearson’s correlation coefficient (group 2): -0.1


<ipython-input-6-f11059d986d6>:39: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [8]:
# (f): Group comparison (4)

# t-test
alpha_mean_1 = theta_opt[0:25, 0]
alpha_mean_2 = theta_opt[25:50, 0]
beta_mean_1 = theta_opt[0:25, 1]
beta_mean_2 = theta_opt[25:50, 1]
t_alpha = stats.ttest_ind(alpha_mean_1, alpha_mean_2)
t_beta = stats.ttest_ind(beta_mean_1, beta_mean_2)

# output
print("The mean of alpha in 2 groups:", round(np.mean(alpha_mean_1), 2), \
      round(np.mean(alpha_mean_2), 2))
print("The t-statistic of alpha:", round(t_alpha[0], 2))
print("The p-value of alpha:", round(t_alpha[1], 2))
print("The mean of beta in 2 groups:", round(np.mean(beta_mean_1), 2), \
      round(np.mean(beta_mean_2), 2))
print("The t-statistic of beta:", round(t_beta[0], 2))
print("The p-value of beta:", round(t_beta[1], 2))
print("Degrees of freedom:", 48) # (n1+n2-2)

The mean of alpha in 2 groups: 0.47 0.38
The t-statistic of alpha: 3.03
The p-value of alpha: 0.0
The mean of beta in 2 groups: 6.03 4.53
The t-statistic of beta: 3.3
The p-value of beta: 0.0
Degrees of freedom: 48


In [9]:
# (g): Parameter recovery (9)

# config
mu_alpha = [np.mean(alpha_mean_1), np.mean(alpha_mean_2)]
mu_beta = [np.mean(beta_mean_1), np.mean(beta_mean_2)]
sigma_alpha = 0.01
sigma_beta = 0.5
V0 = [0.5, 0.5]
n = 1
update = update_1
decision = softmax_decision
repeat_num = 5
participants_num = np.arange(4, 54, 5)
trials_num = np.arange(5, 165, 5)

# recovery
theta_cor_samples = np.zeros([repeat_num, 2])
choices_sample = np.zeros([50, 160])
outcomes_sample = np.zeros([50, 160])
theta_samples_fit = np.zeros([50, 2])
for r in range(repeat_num):
    # sample parameters
    theta_sample = np.zeros([50, 2])
    theta_sample_record = np.zeros([50, 2, 20])
    for k in range(2):
        for i in range(25):
            # ensure alpha and beta in mean+-1.96*std
            j_a = 0
            j_b = 0
            while theta_sample[k*25+i, 0] <= mu_alpha[k] - 1.96 * sigma_alpha \
            or theta_sample[k*25+i, 0] >= mu_alpha[k] + 1.96 * sigma_alpha:
                theta_sample[k*25+i, 0] = np.random.normal(mu_alpha[k], sigma_alpha)
                theta_sample_record[k*25+i, 0, j_a] = theta_sample[k*25+i, 0]
                j_a = j_a + 1
            while theta_sample[k*25+i, 1] <= mu_beta[k] - 1.96 * sigma_beta \
            or theta_sample[k*25+i, 1] >= mu_beta[k] + 1.96 * sigma_beta:
                theta_sample[k*25+i, 1] = np.random.normal(mu_beta[k], sigma_beta)
                theta_sample_record[k*25+i, 1, j_b] = theta_sample[k*25+i, 1]
                j_b = j_b + 1
    # simulation and fit by optimizing NLL and statistics
    for i in range(50):
        choices_sample[i, :], outcomes_sample[i, :] = \
        simulation(V0, n, update, decision, theta_sample[i, 0], \
                   theta_sample[i, 1], sample = True)
        theta_samples_fit[i, :] = \
        minimize(nll, [0.3, 8], args = (choices_sample[i, :], \
                                        outcomes_sample[i, :], V0), \
                 method = "Nelder-Mead")['x']
    theta_cor_samples[r, 0] = np.corrcoef(theta_samples_fit[:, 0], \
                                          theta_sample[:, 0])[0, 1]
    theta_cor_samples[r, 1] = np.corrcoef(theta_samples_fit[:, 1], \
                                          theta_sample[:, 1])[0, 1]

# explore
choices_sample = np.zeros([50, 160])
outcomes_sample = np.zeros([50, 160])
theta_cor_sample = np.zeros([len(participants_num), len(trials_num), 2])
theta_sample_fit = np.zeros([50, len(trials_num), 2])
theta_sample_copy = np.zeros([50, 2])
for i in range(25):
    theta_sample_copy[2*i, :] = theta_sample[i, :]
    theta_sample_copy[2*i+1, :] = theta_sample[25+i, :]
for i in range(50):
    if (i in participants_num) is True:
        i_index = participants_num.tolist().index(i)
    choices_sample[i, :], outcomes_sample[i, :] = \
    simulation(V0, n, update, decision, theta_sample_copy[i, 0], \
               theta_sample_copy[i, 1], sample = True)
    for j in range(len(trials_num)):
        theta_sample_fit[i, j, :] = \
        minimize(nll, [0.3, 8], args = (choices_sample[i, 0:trials_num[j]], \
                                        outcomes_sample[i, 0:trials_num[j]], V0), \
                 method = "Nelder-Mead")['x']
        if (i in participants_num) is True:
            theta_cor_sample[i_index, j, 0] = \
            np.corrcoef(theta_sample_fit[0:(i+1), j, 0], \
                        theta_sample_copy[0:(i+1), 0])[0, 1]
            theta_cor_sample[i_index, j, 1] = \
            np.corrcoef(theta_sample_fit[0:(i+1), j, 1], \
                        theta_sample_copy[0:(i+1), 1])[0, 1]

# plot
# fig, ax =  plt.subplots(1, 1, sharex = True, figsize = (6,3.7))
# label = [r'$Cor(\alpha)$', r'$Cor(\beta)$']
# color = ['#FF9671', '#FF6F91']
# for i in range(5):
#     plt.vlines(x = i+1, ymin = theta_cor_sample[i, -1, -1, 0], \
# ymax = theta_cor_sample[i, -1, -1, 1], color = '#FFC75F', zorder = 1)
# plt.axhline(y = np.mean(theta_cor_sample[:, -1, -1, 0]), xmin = 0, xmax = 6, \
# linestyle = "dashed", color = color[0], zorder = 2)
# plt.axhline(y = np.mean(theta_cor_sample[:, -1, -1, 1]), xmin = 0, xmax = 6, \
# linestyle = "dashed", color = color[1], zorder = 2)
# ax.scatter(np.arange(1, repeat_num+1, 1), theta_cor_sample[:, -1, -1, 0], \
# label = label[0], color = color[0], zorder = 3)
# ax.scatter(np.arange(1, repeat_num+1, 1), theta_cor_sample[:, -1, -1, 1], \
# label = label[1], color = color[1], zorder = 3)
# ax.set_title('Pearson’s correlation between simulated and re-fitted parameters')
# ax.set_xlabel('Trials')
# ax.set_ylabel('Pearson’s correlation')
# ax.legend(loc = 'upper right')
# plt.show()

# plot
fig, ax =  plt.subplots(1, 2, sharey = True, figsize = (8,3.7))
label = [r'$\alpha$', r'$0.1\beta$']
color = ['#FF9671', '#FF6F91']
#for i in range(5):
#    ax[0].vlines(x = i+1, ymin = theta_cor_sample[i, -1, -1, 0], ymax = \
# theta_cor_sample[i, -1, -1, 1], color = '#FFC75F', zorder = 1)
for i in range(2):
    ax[0].axhline(y = mu_alpha[i], xmin = 0.5*i, xmax = 0.5*(i+1), \
                  color = color[0], zorder = 2)
    ax[0].axhline(y = 0.1*mu_beta[i], xmin = 0.5*i, xmax = 0.5*(i+1), \
                  color = color[1], zorder = 2)
ax[0].axhline(y = np.mean(mu_alpha[:]), xmin = 0, xmax = 1, \
              linestyle = "dashed", color = color[0], zorder = 2)
ax[0].axhline(y = 0.1*np.mean(mu_beta[:]), xmin = 0, xmax = 1, \
              linestyle = "dashed", color = color[1], zorder = 2)
ax[0].scatter(np.arange(1, 51, 1), theta_sample[:, 0], label = label[0], \
              color = color[0], zorder = 3)
ax[0].scatter(np.arange(1, 51, 1), 0.1 * theta_sample[:, 1], label = label[1], \
              color = color[1], zorder = 3)
ax[0].axvline(x = 25.5, ymin = 0, ymax = 1, linestyle = "dashed", \
              color = '#FFC75F', zorder = 2)
for i in range (25):
    for k in range(2):
        for j in range(20):
            if (theta_sample_record[25*k+i, 0, j] <= mu_alpha[k] - 1.96 * \
                sigma_alpha or theta_sample_record[25*k+i, 0, j] >= mu_alpha[k] + \
                1.96 * sigma_alpha) and theta_sample_record[25*k+i, 0, j] != 0:
                ax[0].scatter(25*k+i+1, theta_sample_record[25*k+i, 0, j], \
                              color = '#D65DB1', zorder = 3)
            if (theta_sample_record[25*k+i, 1, j] <= mu_beta[k] - 1.96 * sigma_beta \
                or theta_sample_record[25*k+i, 1, j] >= mu_beta[k] + 1.96 * sigma_beta) \
            and theta_sample_record[25*k+i, 1, j] != 0:
                ax[0].scatter(25*k+i+1, 0.1*theta_sample_record[25*k+i, 1, j], \
                              color = '#845EC2', zorder = 3)
ax[0].set_title('Simulated params and outliers')
ax[0].set_xlabel('Participant index')
ax[0].set_ylabel('Simulated parameters')
ax[0].legend(loc = 'upper right')
ax[1].scatter(theta_samples_fit[:, 0], theta_sample[:, 0], label = label[0], \
              color = color[0], zorder = 3)
ax[1].scatter(0.1*theta_samples_fit[:, 1], 0.1*theta_sample[:, 1], label = label[1], \
              color = color[1], zorder = 3)
alpha_min = np.min([np.min(theta_sample[:, 0]), np.min(theta_samples_fit[:, 0])])
beta_min = 0.1 * np.min([np.min(theta_sample[:, 1]), np.min(theta_samples_fit[:, 1])])
alpha_max = np.max([np.max(theta_sample[:, 0]), np.max(theta_samples_fit[:, 0])])
beta_max = 0.1 * np.max([np.max(theta_sample[:, 1]), np.max(theta_samples_fit[:, 1])])
lim_min = np.min([alpha_min, beta_min])
lim_max = np.max([alpha_max, beta_max])
lim_min = np.min([lim_min, 0.1*np.min(theta_sample_record[theta_sample_record>0])]) - 0.05
lim_max = np.max([lim_max, 0.1*np.max(theta_sample_record[theta_sample_record>0])]) + 0.05
x_grid = np.arange(lim_min, lim_max, 0.01)
ax[1].set_ylim(lim_min, lim_max)
ax[1].set_xlim(lim_min, lim_max)
ax[1].plot(x_grid, x_grid, linestyle = "dashed", color = '#FFC75F', zorder = 2)
ax[1].set_title('Cor between simulated and fitted params')
# ax[1].set_ylabel('Simulated parameters')
ax[1].set_xlabel('Fitted parameters')
ax[1].legend(loc = 'upper right')
plt.show()

# image export
# plt.savefig('f4.pgf', format='pgf')

# 3d plots
trials_axis, participants_axis = np.meshgrid(trials_num, participants_num)
participants_axis.resize(len(participants_num)*len(trials_num))
trials_axis.resize(len(participants_num)*len(trials_num))
theta_cor_sample_mean = theta_cor_sample
theta_cor_sample_mean_alpha = \
theta_cor_sample_mean[:, :, 0].reshape(len(participants_num)*len(trials_num))
theta_cor_sample_mean_beta = \
theta_cor_sample_mean[:, :, 1].reshape(len(participants_num)*len(trials_num))

# 3d plot 1
fig = plt.figure(figsize = (6,4.7))
ax = fig.gca(projection = '3d')
plot_g_1 = ax.plot_trisurf(participants_axis, trials_axis, theta_cor_sample_mean_alpha, \
                           cmap = plt.cm.viridis, linewidth = 0, vmin=-0.2, vmax=0.6)
fig.colorbar(plot_g_1)
ax.set_title(r'Pearson’s correlation between simulated and fitted $\alpha$')
ax.set_ylabel('Trials (1 to 5/10/15...)')
ax.set_xlabel('Participants (1 to 2/4/6...)')
plt.show()

# image export
# plt.savefig('s1.pgf', format='pgf')

# 3d plot 2
fig = plt.figure(figsize = (6,4.7))
ax = fig.gca(projection = '3d')
plot_g_2 = ax.plot_trisurf(participants_axis, trials_axis, theta_cor_sample_mean_beta, \
                           cmap = plt.cm.viridis, linewidth = 0, vmin=-0.2, vmax=0.6)
fig.colorbar(plot_g_2)
ax.set_title(r'Pearson’s correlation between simulated and fitted $\beta$')
ax.set_ylabel('Trials (1 to 5/10/15...)')
ax.set_xlabel('Participants (1 to 2/4/6...)')
plt.show()

# image export
# plt.savefig('s2.pgf', format='pgf')

# output
print("Pearson's correlation of alpha:", [round(i,2) for i in theta_cor_samples[:, 0]])
print("Pearson's correlation of beta:", [round(i,2) for i in theta_cor_samples[:, 1]])

<ipython-input-5-8af80401329a>:11: RuntimeWarning: overflow encountered in exp
  nll[i] = 1 / (1 + np.exp(-theta[1] * (V[c_alt-1] - V[c-1])))
<ipython-input-5-8af80401329a>:13: RuntimeWarning: divide by zero encountered in log
  nll = -np.sum(np.log(nll))
<ipython-input-9-e55ae3419970>:123: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-9-e55ae3419970>:144: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Pearson's correlation of alpha: [0.25, 0.38, 0.4, 0.57, 0.43]
Pearson's correlation of beta: [0.7, 0.68, 0.65, 0.73, 0.5]


<ipython-input-9-e55ae3419970>:157: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [10]:
# (h): Alternative model (9)

# update strategy 2
def update_2(V, action, outcome, alpha):
    V[action-1] = alpha[0] * V[action-1] + alpha[1] * (outcome - V[action-1])
    return V

# simulation() remains the same as before by setting the config:
# update = update_2
# alpha = [alpha, A]

# negative log likelihood
def nll_A(theta, choices, outcomes, V0):
    # theta = [alpha, beta, A]
    length_num = len(choices)
    V = copy.deepcopy(V0)
    nll_A = np.zeros(length_num)
    for i in range(length_num):
        c = int(choices[i])
        c_alt = 1 + int(c==1)
        nll_A[i] = 1 / (1 + np.exp(-theta[1] * (V[c_alt-1] - V[c-1])))
        V[c-1] = theta[2] * V[c-1] + theta[0] * (int(outcomes[i]) - V[c-1])
    nll_A = -np.sum(np.log(nll_A))
    return nll_A

# optimization
V0 = [0.5, 0.5]
theta_opt_A = np.zeros([50,3])
nll_A_obj = np.zeros(50)
for i in range(50):    
    nll_A_result = minimize(nll_A, [0.4, 5, 0.5], \
                            args = (choices.loc[i], outcomes.loc[i], V0), \
                            method = "Nelder-Mead")
    theta_opt_A[i, :] = nll_A_result['x']
    nll_A_obj[i] = nll_A_result['fun']

# negative log likelihood
def nll_A1(theta, choices, outcomes, V0):
    # theta = [alpha, beta, np.log(np.divide(1, A) - 1)]
    length_num = len(choices)
    V = copy.deepcopy(V0)
    nll_A1 = np.zeros(length_num)
    for i in range(length_num):
        c = int(choices[i])
        c_alt = 1 + int(c==1)
        nll_A1[i] = 1 / (1 + np.exp(-theta[1] * (V[c_alt-1] - V[c-1])))
        V[c-1] = V[c-1] / (1 + np.exp(theta[2])) + theta[0] * \
        (int(outcomes[i]) - V[c-1])
    nll_A1 = -np.sum(np.log(nll_A1))
    return nll_A1

# optimization
V0 = [0.5, 0.5]
theta_opt_A1 = np.zeros([50,3])
for i in range(50):    
    theta_opt_A1[i, :] = minimize(nll_A1, [0.4, 5, np.log(np.divide(1, 0.5) - 1)], \
                                  args = (choices.loc[i], outcomes.loc[i], V0), \
                                  method = "Nelder-Mead")['x']
theta_opt_A1[:, 2] = 1 / (1 + np.exp(theta_opt_A1[:, 2]))

# statistics
theta_mean_A = np.mean(theta_opt_A, axis = 0)
theta_var_A = np.var(theta_opt_A, axis = 0)
theta_mean_A1 = np.mean(theta_opt_A1, axis = 0)
theta_var_A1 = np.var(theta_opt_A1, axis = 0)
theta_cor_A1 = np.corrcoef(theta_opt_A1[:, 0], theta_opt_A1[:, 2])[0, 1]

# plot
theta_opt_A1[:, 1] = 0.1 * theta_opt_A1[:, 1] # warning
fig, ax =  plt.subplots(1, 2, figsize = (8,3.7))
label_A = [r'$\alpha$', r'$\beta$', 'A']
label_A1 = [r'$\alpha$', r'0.1$\beta$', 'A']
color = ['#FF9671', '#FF6F91', '#D65DB1']
# color = ['#FF6F91', '#D65DB1']
for i in range(50):
    ax[0].vlines(x = i+1, ymin = np.min(theta_opt_A[i, :]), ymax = \
                 np.max(theta_opt_A[i, :]), color = '#FFC75F', zorder = 1)
    ax[1].vlines(x = i+1, ymin = np.min(theta_opt_A1[i, :]), ymax = \
                 np.max(theta_opt_A1[i, :]), color = '#FFC75F', zorder = 1)
for i in range(2):
    ax[i].axvline(x = 25.5, ymin = 0, ymax = 1, linestyle = "dashed", \
                  color = '#FFC75F', zorder = 2)
for i in range(3):
    ax[0].scatter(np.arange(1, 51, 1), theta_opt_A[:, i], label = label_A[i], \
                  color = color[i], zorder = 3)
    ax[1].scatter(np.arange(1, 51, 1), theta_opt_A1[:, i], label = label_A1[i], \
                  color = color[i], zorder = 3)
ax[0].set_xlabel('Participant index')
ax[1].set_xlabel('Participant index')
ax[0].set_ylabel('Parameter value')
ax[0].legend(loc = 'upper right')
ax[1].legend(loc = 'upper right')
ax[0].set_title('Unrestricted A')
ax[1].set_title('Restricted A ($=1/(1+e^{A^{*}})$)')
# plt.suptitle('Parameter optimization of NLL by Nelder-Mead')
theta_opt_A1[:, 1] = 10 * theta_opt_A1[:, 1] # warning
plt.show()

# output
print("Mean of fitted parameter values [alpha, beta, A] by NLL(A):", \
      [round(i,2) for i in theta_mean_A])
print("Mean of fitted parameter values [alpha, beta, A] by restricted NLL(A):", \
      [round(i,2) for i in theta_mean_A1])
print("Variance of fitted parameter values [alpha, beta, A] by NLL(A):", \
      [round(i,2) for i in theta_var_A])
print("Variance of fitted parameter values [alpha, beta, A] by restricted NLL(A):", \
      [round(i,2) for i in theta_var_A1])
print("Pearson’s correlation coefficient (total) between alpha and A:", \
      round(theta_cor_A1,2))

# image export
# plt.savefig('f5.pgf', format='pgf')

<ipython-input-10-8f618d34b796>:21: RuntimeWarning: overflow encountered in exp
  nll_A[i] = 1 / (1 + np.exp(-theta[1] * (V[c_alt-1] - V[c-1])))
<ipython-input-10-8f618d34b796>:23: RuntimeWarning: divide by zero encountered in log
  nll_A = -np.sum(np.log(nll_A))


Mean of fitted parameter values [alpha, beta, A] by NLL(A): [8.29, 2.98, 8.88]
Mean of fitted parameter values [alpha, beta, A] by restricted NLL(A): [0.4, 5.68, 0.98]
Variance of fitted parameter values [alpha, beta, A] by NLL(A): [123.44, 10.89, 123.83]
Variance of fitted parameter values [alpha, beta, A] by restricted NLL(A): [0.02, 4.66, 0.0]
Pearson’s correlation coefficient (total) between alpha and A: 0.29


<ipython-input-10-8f618d34b796>:87: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [11]:
# (k): Discussion and extra model (15)

# update strategy 3
def update_3(V, action, outcome, alpha):
    V[action-1] = V[action-1] + ((1 - outcome) * alpha[0] + outcome * alpha[1]) \
    * (outcome - V[action-1])
    return V

# simulation() remains the same as before by setting the config:
# update = update_3
# alpha = [alpha_pos, alpha_neg]

# negative log likelihood
def nll_aa(theta, choices, outcomes, V0):
    # theta = [alpha_pos, alpha_neg, beta]
    length_num = len(choices)
    V = copy.deepcopy(V0)
    nll_aa = np.zeros(length_num)
    for i in range(length_num):
        c = int(choices[i])
        c_alt = 1 + int(c==1)
        nll_aa[i] = 1 / (1 + np.exp(-theta[2] * (V[c_alt-1] - V[c-1])))
        V[c-1] = V[c-1] + ((1 - int(outcomes[i])) * theta[0] + int(outcomes[i]) \
                           * theta[1]) * (int(outcomes[i]) - V[c-1])
    nll_aa = -np.sum(np.log(nll_aa))
    return nll_aa

# optimization
V0 = [0.5, 0.5]
theta_opt_aa = np.zeros([50,3])
nll_aa_obj = np.zeros(50)
for i in range(50):    
    nll_aa_result = minimize(nll_aa, [0.25, 0.35, 6], \
                             args = (choices.loc[i], outcomes.loc[i], V0), \
                             method = "Nelder-Mead")
    theta_opt_aa[i, :] = nll_aa_result['x']
    nll_aa_obj[i] = nll_aa_result['fun']

# statistics
theta_mean_aa = np.mean(theta_opt_aa, axis = 0)
theta_var_aa = np.var(theta_opt_aa, axis = 0)
a_pos_mean_1 = theta_opt_aa[0:25, 0]
a_pos_mean_2 = theta_opt_aa[25:50, 0]
a_neg_mean_1 = theta_opt_aa[0:25, 1]
a_neg_mean_2 = theta_opt_aa[25:50, 1]
beta_aa_mean_1 = theta_opt_aa[0:25, 2]
beta_aa_mean_2 = theta_opt_aa[25:50, 2]
t_a_pos = stats.ttest_ind(a_pos_mean_1, a_pos_mean_2)
t_a_neg = stats.ttest_ind(a_neg_mean_1, a_neg_mean_2)
t_beta_aa = stats.ttest_ind(beta_aa_mean_1, beta_aa_mean_2)

# plot
theta_opt_aa[:, 2] = 0.1 * theta_opt_aa[:, 2] # warning
fig, ax =  plt.subplots(1, 2, figsize = (8,3.7))
label_aa = [r'$\alpha^+$', r'$\alpha^-$', r'0.1$\beta$']
color = ['#FF9671', '#FF6F91', '#D65DB1']
for i in range(50):
    for j in range(2):
        ax[j].vlines(x = i+1, ymin = np.min(theta_opt_aa[i, :]), \
                     ymax = np.max(theta_opt_aa[i, :]), color = '#FFC75F', zorder = 1)
for i in range(3):
    for j in range(2):
        ax[j].scatter(np.arange(1, 51, 1), theta_opt_aa[:, i], \
                      label = label_aa[i], color = color[i], zorder = 3)
        ax[j].axhline(y = np.mean(theta_opt_aa[:, i]), xmin = 0, xmax = 1, \
                      linestyle = "dashed", color = color[i], zorder = 2)
        for k in range(2):
            ax[j].axhline(y = np.mean(theta_opt_aa[(25*k):(25*(k+1)), i]), \
                          xmin = 0.5*k, xmax = 0.5*(k+1), color = color[i], zorder = 2)
for i in range(2):
    ax[i].axvline(x = 25.5, ymin = 0, ymax = 1, linestyle = "dashed", \
                  color = '#FFC75F', zorder = 2)
    ax[i].set_xlabel('Participant index')
    ax[i].set_ylabel('Parameter value')
    ax[i].legend(loc = 'upper right')
    ax[i].set_xlabel('Participant index')
ax[0].set_title(r'Model with NLL($\alpha^{\pm}$)')
ax[1].set_title('Specified range')
ax[1].set_ylim(-0.1, 1.6)
theta_opt_aa[:, 2] = 10 * theta_opt_aa[:, 2] # warning
plt.show()

# output
print("Mean of fitted parameter values [alpha+, alpha-, beta] by NLL(aa):", \
      [round(i, 2) for i in theta_mean_aa])
print("Variance of fitted parameter values [alpha+, alpha-, beta] by NLL(aa):", \
      [round(i, 2) for i in theta_var_aa])
print("Mean of fitted parameter values [alpha+, alpha-, beta] in group 1:", \
      round(np.mean(a_pos_mean_1), 2), round(np.mean(a_neg_mean_1), 2), \
      round(np.mean(beta_aa_mean_1), 2))
print("Mean of fitted parameter values [alpha+, alpha-, beta] in group 2:", \
      round(np.mean(a_pos_mean_2), 2), round(np.mean(a_neg_mean_2), 2), \
      round(np.mean(beta_aa_mean_2), 2))
print("The t-statistic of alpha+:", round(t_a_pos[0], 2))
print("The p-value of alpha+:", round(t_a_pos[1], 2))
print("The t-statistic of alpha-:", round(t_a_neg[0], 2))
print("The p-value of alpha-:", round(t_a_neg[1], 2))
print("The t-statistic of beta:", round(t_beta_aa[0], 2))
print("The p-value of beta:", round(t_beta_aa[1], 2))

# image export
# plt.savefig('f6.pgf', format='pgf')

<ipython-input-11-da62cae4bf68>:72: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Mean of fitted parameter values [alpha+, alpha-, beta] by NLL(aa): [0.26, 0.52, 11.49]
Variance of fitted parameter values [alpha+, alpha-, beta] by NLL(aa): [0.03, 0.02, 188.12]
Mean of fitted parameter values [alpha+, alpha-, beta] in group 1: 0.29 0.58 14.8
Mean of fitted parameter values [alpha+, alpha-, beta] in group 2: 0.23 0.47 8.18
The t-statistic of alpha+: 0.98
The p-value of alpha+: 0.33
The t-statistic of alpha-: 2.84
The p-value of alpha-: 0.01
The t-statistic of beta: 1.72
The p-value of beta: 0.09


In [12]:
# (i): Model comparison (8)

# plot
fig, ax =  plt.subplots(1, 1, sharex = True, figsize = (7,3.7))
label = ['Model 1', 'Model 2', 'Model 3']
color = ['#FF9671', '#FF6F91', '#D65DB1']
ax.scatter(np.arange(1, 51, 1), nll_a_obj, label = label[0], \
           color = color[0], zorder = 3)
ax.scatter(np.arange(1, 51, 1), nll_A_obj, label = label[1], \
           color = color[1], zorder = 3)
ax.scatter(np.arange(1, 51, 1), nll_aa_obj, label = label[2], \
           color = color[2], zorder = 3)
for i in range(50):
    plt.vlines(x = i+1, ymin = np.min([nll_a_obj[i], nll_A_obj[i], nll_aa_obj[i]]), \
               ymax = np.max([nll_a_obj[i], nll_A_obj[i], nll_aa_obj[i]]), \
               color = '#FFC75F', zorder = 1)
plt.axvline(x = 25.5, ymin = 0, ymax = 1, linestyle = "dashed", \
            color = '#FFC75F', zorder = 2)
plt.axhline(y = np.mean(nll_a_obj), xmin = 0, xmax = 1, \
            linestyle = "dashed", color = color[0], zorder = 2)
plt.axhline(y = np.mean(nll_A_obj), xmin = 0, xmax = 1, \
            linestyle = "dashed", color = color[1], zorder = 2)
plt.axhline(y = np.mean(nll_aa_obj), xmin = 0, xmax = 1, \
            linestyle = "dashed", color = color[2], zorder = 2)
plt.axhline(y = np.mean(nll_a_obj[0:25]), xmin = 0, xmax = 0.5, \
            color = color[0], zorder = 2)
plt.axhline(y = np.mean(nll_A_obj[0:25]), xmin = 0, xmax = 0.5, \
            color = color[1], zorder = 2)
plt.axhline(y = np.mean(nll_aa_obj[0:25]), xmin = 0, xmax = 0.5, \
            color = color[2], zorder = 2)
plt.axhline(y = np.mean(nll_a_obj[25:50]), xmin = 0.5, xmax = 1, \
            color = color[0], zorder = 2)
plt.axhline(y = np.mean(nll_A_obj[25:50]), xmin = 0.5, xmax = 1, \
            color = color[1], zorder = 2)
plt.axhline(y = np.mean(nll_a_obj[25:50]), xmin = 0.5, xmax = 1, \
            color = color[2], zorder = 2)
ax.set_title('NLL values')
ax.set_xlabel('Participant index')
ax.set_ylabel('NLL objective function values')
ax.legend(loc = 'upper right')
plt.show()

# AIC and BIC
aic_nll_a = np.sum(2 * nll_a_obj + 2 * 2)
aic_nll_A = np.sum(2 * nll_A_obj + 2 * 3)
aic_nll_aa = np.sum(2 * nll_aa_obj + 2 * 3)
aic_nll_a_1 = np.sum(2 * nll_a_obj[0:25] + 2 * 2)
aic_nll_A_1 = np.sum(2 * nll_A_obj[0:25] + 2 * 3)
aic_nll_aa_1 = np.sum(2 * nll_aa_obj[0:25] + 2 * 3)
aic_nll_a_2 = np.sum(2 * nll_a_obj[25:50] + 2 * 2)
aic_nll_A_2 = np.sum(2 * nll_A_obj[25:50] + 2 * 3)
aic_nll_aa_2 = np.sum(2 * nll_aa_obj[25:50] + 2 * 3)
bic_nll_a = np.sum(2 * nll_a_obj + 2 * np.log(160))
bic_nll_A = np.sum(2 * nll_A_obj + 3 * np.log(160))
bic_nll_aa = np.sum(2 * nll_aa_obj + 3 * np.log(160))
bic_nll_a_1 = np.sum(2 * nll_a_obj[0:25] + 2 * np.log(160))
bic_nll_A_1 = np.sum(2 * nll_A_obj[0:25] + 3 * np.log(160))
bic_nll_aa_1 = np.sum(2 * nll_aa_obj[0:25] + 3 * np.log(160))
bic_nll_a_2 = np.sum(2 * nll_a_obj[25:50] + 2 * np.log(160))
bic_nll_A_2 = np.sum(2 * nll_A_obj[25:50] + 3 * np.log(160))
bic_nll_aa_2 = np.sum(2 * nll_aa_obj[25:50] + 3 * np.log(160))

# output
print("AIC of NLL(a):", round(aic_nll_a, 2), \
      round(aic_nll_a_1, 2), round(aic_nll_a_2, 2))
print("AIC of NLL(A):", round(aic_nll_A, 2), \
      round(aic_nll_A_1, 2), round(aic_nll_A_2, 2))
print("AIC of NLL(aa):", round(aic_nll_aa, 2), \
      round(aic_nll_aa_1, 2), round(aic_nll_aa_2, 2))
print("BIC of NLL(a):", round(bic_nll_a, 2), \
      round(aic_nll_a_1, 2), round(aic_nll_a_2, 2))
print("BIC of NLL(A):", round(bic_nll_A, 2), \
      round(aic_nll_A_1, 2), round(aic_nll_A_2, 2))
print("BIC of NLL(aa):", round(bic_nll_aa, 2), \
      round(aic_nll_aa_1, 2), round(aic_nll_aa_2, 2))

# image export
# plt.savefig('s3.pgf', format='pgf')

AIC of NLL(a): 6070.43 2601.82 3468.62
AIC of NLL(A): 6123.36 2628.36 3495.01
AIC of NLL(aa): 6043.09 2591.55 3451.54
BIC of NLL(a): 6377.95 2601.82 3468.62
BIC of NLL(A): 6584.64 2628.36 3495.01
BIC of NLL(aa): 6504.37 2591.55 3451.54


<ipython-input-12-d02f7444c640>:26: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [152]:
# (j) Model recovery and confusion matrix (10)

# config
theta_mean_set = np.zeros([6, 3])
theta_std_set = np.zeros([6, 3])
range_025_975 = np.zeros([6, 3, 2])
# 95% interval in normal distribution, excluding extreme samples

for i in range(2):
    theta_mean_set[i, 0:2] = np.mean(theta_opt[(25*i):(25*(i+1)), :], \
                                     axis = 0) # group i mean[alpha, beta]
    theta_std_set[i, 0:2] = np.std(theta_opt[(25*i):(25*(i+1)), :], \
                                   axis = 0) # group i std[alpha, beta]
    theta_mean_set[i+2, :] = np.mean(theta_opt_A[(25*i):(25*(i+1)), :], \
                                     axis = 0) # group i mean[alpha, beta, A]
    theta_std_set[i+2, :] = np.std(theta_opt_A[(25*i):(25*(i+1)), :], \
                                   axis = 0) # group i std[alpha, beta, A]
    theta_mean_set[i+4, :] = np.mean(theta_opt_aa[(25*i):(25*(i+1)), :], \
                                     axis = 0) # group i mean[alpha+, alpha-, beta]
    theta_std_set[i+4, :] = np.std(theta_opt_aa[(25*i):(25*(i+1)), :], \
                                   axis = 0) # group i std[alpha+, alpha-, beta]
for i in range(6):
    range_025_975[i, :, 0] = theta_mean_set[i, :] - 1.96 * theta_std_set[i, :]
    range_025_975[i, :, 1] = theta_mean_set[i, :] + 1.96 * theta_std_set[i, :]
theta_alpha_index = [[0], [0, 2], [0, 1]]
theta_beta_index = [[1], [1], [2]]
param_num_3 = [2, 3, 3]
param_num_6 = [2, 2, 3, 3, 3, 3]
V0 = [0.5, 0.5]
n = 1
update_set = [update_1, update_2, update_3]
decision = softmax_decision
subsample_size = 250
nll_set = [nll, nll_A, nll_aa]
theta_init_set = [np.mean(theta_opt, axis = 0), np.mean(theta_opt_A, axis = 0), \
                  np.mean(theta_opt_aa, axis = 0)]

# sample parameters
theta_subsample = np.zeros([6, 3, subsample_size])
for k in range(6):
    for i in range(param_num_6[k]):
        for j in range(subsample_size):
            while theta_subsample[k, i, j] <= range_025_975[k, i, 0] or \
            theta_subsample[k, i, j] >= range_025_975[k, i, 1] or \
            theta_subsample[k, i, j] == 0:
                theta_subsample[k, i, j] = np.random.normal(theta_mean_set[k, i], \
                                                            theta_std_set[k, i])

# simulation and fit by optimizing 3 different NLLs and statistics
nll_values_3 = np.zeros([3, 3, 2*subsample_size]) # [nll_true, nll_fit, num]
for p in range(3):
    for q in range(2):
        for i in range(subsample_size):
            choices_subsample, outcomes_subsample = \
            simulation(V0, n, update_set[p], decision, \
                       theta_subsample[2*p+q, theta_alpha_index[p], i], \
                       theta_subsample[2*p+q, theta_beta_index[p], i], sample = True)
            for k in range(3):
                nll_values_3[p, k, subsample_size*q+i] = \
                minimize(nll_set[k], theta_init_set[k], args = \
                         (choices_subsample, outcomes_subsample, V0), \
                         method = "Nelder-Mead")['fun']

# AIC and BIC
aic_nll_3 = np.zeros([3, 3, 2*subsample_size])
bic_nll_3 = np.zeros([3, 3, 2*subsample_size])
for i in range(3):
    aic_nll_3[:, i, :] = 2 * nll_values_3[:, i, :] + 2 * param_num_3[i]
    bic_nll_3[:, i, :] = 2 * nll_values_3[:, i, :] + param_num_3[i] * np.log(160)

# confusion matrix
aic_cm = np.zeros([3, 3])
bic_cm = np.zeros([3, 3])
for i in range(3):
    for j in range(subsample_size):
        aic_win_index = np.argmin(aic_nll_3[i, :, j])
        aic_cm[i, aic_win_index] = aic_cm[i, aic_win_index] + 1
        bic_win_index = np.argmin(bic_nll_3[i, :, j])
        bic_cm[i, bic_win_index] = bic_cm[i, bic_win_index] + 1
aic_cm = np.round(aic_cm / subsample_size, 2)
bic_cm = np.round(bic_cm / subsample_size, 2)

# t-test
alpha_pos_mean_1 = theta_opt_aa[0:25, 0]
alpha_pos_mean_2 = theta_opt_aa[25:50, 0]
alpha_neg_mean_1 = theta_opt_aa[0:25, 1]
alpha_neg_mean_2 = theta_opt_aa[25:50, 1]
beta_mean_1 = theta_opt_aa[0:25, 2]
beta_mean_2 = theta_opt_aa[25:50, 2]
t_alpha_pos = stats.ttest_ind(alpha_pos_mean_1, alpha_pos_mean_2)
t_alpha_neg = stats.ttest_ind(alpha_neg_mean_1, alpha_neg_mean_2)
t_beta = stats.ttest_ind(beta_mean_1, beta_mean_2)

# output
print("Confusion matrix under AIC is:")
print(aic_cm)
print("Confusion matrix under BIC is:")
print(bic_cm)
print("The t-statistic of alpha+:", round(t_alpha_pos[0], 2))
print("The p-value of alpha+:", round(t_alpha_pos[1], 2))
print("The t-statistic of alpha-:", round(t_alpha_neg[0], 2))
print("The p-value of alpha-:", round(t_alpha_neg[1], 2))
print("The t-statistic of beta:", round(t_beta[0], 2))
print("The p-value of beta:", round(t_beta[1], 2))
print("Degrees of freedom:", 47) # (n1+n2-3)

<ipython-input-10-e10583a7e47c>:21: RuntimeWarning: overflow encountered in exp
  nll_A[i] = 1 / (1 + np.exp(-theta[1] * (V[c_alt-1] - V[c-1])))
<ipython-input-10-e10583a7e47c>:23: RuntimeWarning: divide by zero encountered in log
  nll_A = -np.sum(np.log(nll_A))
<ipython-input-5-8af80401329a>:11: RuntimeWarning: overflow encountered in exp
  nll[i] = 1 / (1 + np.exp(-theta[1] * (V[c_alt-1] - V[c-1])))
<ipython-input-5-8af80401329a>:13: RuntimeWarning: divide by zero encountered in log
  nll = -np.sum(np.log(nll))
<ipython-input-3-27bd0dce688c>:10: RuntimeWarning: overflow encountered in exp
  p_action_A = 1 / (1 + np.exp(-beta * (V[1] - V[0])))
<ipython-input-10-e10583a7e47c>:22: RuntimeWarning: overflow encountered in double_scalars
  V[c-1] = theta[2] * V[c-1] + theta[0] * (int(outcomes[i]) - V[c-1])


Confusion matrix under AIC is:
[[0.66 0.1  0.24]
 [0.51 0.2  0.29]
 [0.43 0.12 0.45]]
Confusion matrix under BIC is:
[[0.92 0.01 0.06]
 [0.64 0.16 0.2 ]
 [0.74 0.04 0.22]]
The t-statistic of alpha+: 0.98
The p-value of alpha+: 0.33
The t-statistic of alpha-: 2.84
The p-value of alpha-: 0.01
The t-statistic of beta: 1.72
The p-value of beta: 0.09
Degrees of freedom: 47
